In [1]:
import os
os.chdir('../')

In [6]:
import data
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()

In [25]:
train = data.train_df('small')

In [26]:
train.head(15)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
17199,08X04F6M86EW,f4e3a191fe69f,1541452894,1,search for destination,"Cusco, Peru",BR,"Cusco, Peru",desktop,NaN,NaN,NaN,1
17200,08X04F6M86EW,f4e3a191fe69f,1541452926,2,interaction item rating,2128794,BR,"Cusco, Peru",desktop,NaN,NaN,NaN,1
17201,08X04F6M86EW,f4e3a191fe69f,1541452943,3,clickout item,NaN,BR,"Cusco, Peru",desktop,NaN,2128794|5050054|5050464|4665594|102985|102971|...,146|66|63|8|79|102|54|85|76|65|30|366|71|66|62...,1
17202,0BO8K084XKYE,ad89aba9bb002,1541457675,1,search for item,1111123,US,"Ledyard, USA",tablet,NaN,NaN,NaN,1
17203,0BO8K084XKYE,ad89aba9bb002,1541458697,3,interaction item image,9979396,US,"Ledyard, USA",tablet,NaN,NaN,NaN,2
17204,0BO8K084XKYE,ad89aba9bb002,1541458805,5,clickout item,NaN,US,"Ledyard, USA",tablet,NaN,76028|70388|1336053|317126|79506|9979396|71272...,81|70|104|153|100|86|121|113|113|94|39|94|75|4...,2
17215,0CI40GAHP96Q,74986cb514e04,1541456132,1,search for destination,"Rio Quente, Brazil",BR,"Rio Quente, Brazil",mobile,NaN,NaN,NaN,1
17216,0CI40GAHP96Q,74986cb514e04,1541456168,2,clickout item,8332666,BR,"Rio Quente, Brazil",mobile,NaN,1668387|2260908|3926372|2343438|2571941|257499...,49|30|31|60|354|280|33|50|307|56|64|273|52|27|40,1
17217,0CI40GAHP96Q,74986cb514e04,1541456747,3,search for destination,"Porto Seguro, Brazil",BR,"Porto Seguro, Brazil",mobile,NaN,NaN,NaN,1
17218,0CI40GAHP96Q,74986cb514e04,1541456767,4,search for destination,"Rio Quente, Brazil",BR,"Rio Quente, Brazil",mobile,NaN,NaN,NaN,1


## Filter the sessions the have 0 clickouts or the clickout reference is not in the impressions

In [59]:
def filter_sessions(g):
    # check if there is at least 1 clickout
    clickouts = g[g.action_type == 'clickout item']
    if clickouts.shape[0] > 0:
        # and the last clickout reference is in the impressions
        last_clickout = clickouts.iloc[[-1]]
        imprs = last_clickout.impressions.str.split('|').values[0]
        ref = last_clickout.reference
        if ref.isnull().values[0] or ref.values[0] in imprs:
            return g

In [60]:
df = train.reset_index().groupby(['user_id','session_id']).progress_apply(filter_sessions)

In [62]:
df = df.set_index('index').sort_index()

In [63]:
df

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
index,,,,,,,,,,,,,
17199,08X04F6M86EW,f4e3a191fe69f,1541452894,1,search for destination,"Cusco, Peru",BR,"Cusco, Peru",desktop,NaN,NaN,NaN,1
17200,08X04F6M86EW,f4e3a191fe69f,1541452926,2,interaction item rating,2128794,BR,"Cusco, Peru",desktop,NaN,NaN,NaN,1
17201,08X04F6M86EW,f4e3a191fe69f,1541452943,3,clickout item,NaN,BR,"Cusco, Peru",desktop,NaN,2128794|5050054|5050464|4665594|102985|102971|...,146|66|63|8|79|102|54|85|76|65|30|366|71|66|62...,1
17202,0BO8K084XKYE,ad89aba9bb002,1541457675,1,search for item,1111123,US,"Ledyard, USA",tablet,NaN,NaN,NaN,1
17203,0BO8K084XKYE,ad89aba9bb002,1541458697,3,interaction item image,9979396,US,"Ledyard, USA",tablet,NaN,NaN,NaN,2
17204,0BO8K084XKYE,ad89aba9bb002,1541458805,5,clickout item,NaN,US,"Ledyard, USA",tablet,NaN,76028|70388|1336053|317126|79506|9979396|71272...,81|70|104|153|100|86|121|113|113|94|39|94|75|4...,2
17215,0CI40GAHP96Q,74986cb514e04,1541456132,1,search for destination,"Rio Quente, Brazil",BR,"Rio Quente, Brazil",mobile,NaN,NaN,NaN,1
17216,0CI40GAHP96Q,74986cb514e04,1541456168,2,clickout item,8332666,BR,"Rio Quente, Brazil",mobile,NaN,1668387|2260908|3926372|2343438|2571941|257499...,49|30|31|60|354|280|33|50|307|56|64|273|52|27|40,1
17217,0CI40GAHP96Q,74986cb514e04,1541456747,3,search for destination,"Porto Seguro, Brazil",BR,"Porto Seguro, Brazil",mobile,NaN,NaN,NaN,1


In [61]:
print('Old shape:', train.shape)
print('New shape:', df.shape)

Old shape: (79581, 13)
New shape: (73223, 13)


In [62]:
df.index.values

array([     0,      1,      2, ..., 100026, 100027, 100028])